In [ ]:
# load libraries
import defs
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time as t

In [ ]:
# set plots
%matplotlib inline
 
plt.rcParams['figure.figsize']=(20,10)
plt.style.use('ggplot')

In [ ]:
# get tickers
tickers = []
manager = mp.Manager()
tickers = manager.list()

#get file object
f = open("list", "r")

while(True):
    #read next line
    line = f.readline()
    #if line is empty, you are done with all lines in the file
    if not line:
        break
    #you can access the line
    tickers.append(line.strip())

#close file
f.close()

In [ ]:
# calculates

market_dfset=manager.dict()
modelset=manager.dict()
forecastset=manager.dict()

p = {}
for ticker in tickers:
    p[ticker]= mp.Process(target=defs.run_neuralprophet,args=(tickers,ticker,market_dfset,modelset,forecastset))
    
l = len(tickers)
c = mp.cpu_count()
for i in range(0, l, c):
    for j in range(0,c):
        if (i+j<l):
            p[tickers[i+j]].start()
    for j in range(0,c):
        t.sleep(0.2)
        if (i+j<l): p[tickers[i+j]].join()

In [ ]:
# show graphs
figureset = {}
legendset = {}
for ticker in tickers:
    print(ticker)
    figureset[ticker] = market_dfset[ticker]["Close"].plot()
    legendset[ticker]=figureset[ticker].legend() #get the legend
    legendset[ticker].get_texts()[0].set_text(ticker) #change the legend text
    plt.show()

In [ ]:
for ticker in tickers:
    n=3650
    print(ticker)
    figure=modelset[ticker].plot(forecastset[ticker])
    fig_param = modelset[ticker].plot_parameters()
    plt.show()
    print('\n')

In [ ]:
# merge the sets
full_dfset = {}
for ticker in tickers:
    full_dfset[ticker] = forecastset[ticker].set_index('ds').join(market_dfset[ticker])

In [ ]:
# plot the future
for ticker in tickers:
    print(ticker)
    modelset[ticker].plot(forecastset[ticker].tail(n=60))
    plt.show()
    last = 30
    yhat = []
    for i in range(1,last):
        l = len(full_dfset[ticker])
        hat = 'yhat'+str(i)
        yhat.insert(i-1, full_dfset[ticker][hat][l-last+i-1])
    print(max(yhat))
    print(min(yhat))
    print('\n')

In [ ]:
# write out
for ticker in tickers:
    n = 30
    full_dfset[ticker].tail(n).to_csv(ticker+'-'+str(n)+'-nprophet-Days.csv')